In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
from repository import get_trending_playlist_data

In [1]:
data = get_trending_playlist_data()

Access token obtained successfully.


In [98]:
data.head()

,Track Name,Album Name,Artists,Track ID,Album ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,redrum,american dream,21 Savage,52eIcoLUM25zbQupAZYoFh,2RRYaYHY7fIIdvFlvgb5vq,94,2024-01-12,270698,True,https://open.spotify.com/track/52eIcoLUM25zbQu...,...,0.740,2,-8.445,1,0.0481,0.00529,0.000224,0.5000,0.246,172.089
1,Lovin On Me,Lovin On Me,Jack Harlow,4xhsWYTOGcal8zt0J161CU,6VCO0fDBGbRW8mCEvV95af,94,2023-11-10,138411,True,https://open.spotify.com/track/4xhsWYTOGcal8zt...,...,0.558,2,-4.911,1,0.0568,0.00260,0.000002,0.0937,0.606,104.983
2,BELLAKEO,BELLAKEO,"Peso Pluma, Anitta",05WVKTdZhlIMX4qqMLuo0f,3VLY9g3CAG1Y5r2eGVEaZ0,90,2023-12-07,197333,True,https://open.spotify.com/track/05WVKTdZhlIMX4q...,...,0.880,9,-2.834,1,0.1010,0.05620,0.060000,0.1530,0.463,180.011
3,I'm Good (Blue),I'm Good (Blue),"David Guetta, Bebe Rexha",4uUG5RXrOk84mYEfFvj3cK,7M842DMhYVALrXsw3ty7B3,90,2022-08-26,175238,True,https://open.spotify.com/track/4uUG5RXrOk84mYE...,...,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040
4,Prada,Prada,"cassö, RAYE, D-Block Europe",59NraMJsLaMCVtwXTSia8i,5MU0RmBSpoSxOPYBfcobDc,89,2023-08-11,132359,True,https://open.spotify.com/track/59NraMJsLaMCVtw...,...,0.717,8,-5.804,1,0.0375,0.00100,0.000002,0.1130,0.422,141.904


In [4]:
data.isnull().sum()

Track Name          0
Album Name          0
Artists             0
Track ID            0
Album ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

### Note:
It is important to recommend new releases to users. For that we give more weight to new release

In [139]:
# Function to calculate weighted popularity based on release date
def  calculate_weight_popularity(release_date):
    # Convert release date to datetime
    release_date = datetime.strptime(release_date, '%Y-%m-%d')


    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity based on time span (e.g, more recent release have higher popularity)
    weight = 1 / (time_span.days + 1)
    return weight

In [140]:
# Normalize the music features using Min-max scaling
scaler = MinMaxScaler()
music_feature = data[['Danceability', 'Energy', 'Key', 
                        'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                        'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_feature_scaled = scaler.fit_transform(music_feature)

In [143]:
# Function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in data['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name")
        return
    
    # Get index of input_song_name from DataFrame
    input_song_index = data[data['Track Name'] == input_song_name].index[0]

    # Calculate similarity score based on music Features
    similarity_score = cosine_similarity([music_feature_scaled[input_song_index]], music_feature_scaled)
    
    # Get the indices of most similar score
    similar_score_indices = similarity_score.argsort()[0][::-1][1:num_recommendations+1]

    # Get song name based on indices of similar song
    content_based_recommendations = data.iloc[similar_score_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]
    return content_based_recommendations

In [154]:
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in data['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name")
        return
    
    # Get content based recommendations
    content_based_recd = content_based_recommendations(input_song_name, num_recommendations)

    # Get popularity score of input song
    popularity_score = data.loc[data['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weight_popularity(data.loc[data['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity score
    hybrid_recommendations = content_based_recd
    hybrid_recommendations = hybrid_recommendations._append({
        'Track Name': input_song_name,
        'Artists': data.loc[data['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': data.loc[data['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': data.loc[data['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score,
    }, ignore_index=True)

    # Sort the hybrid recommendation based on weighted popularity
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]
    
    return hybrid_recommendations

In [155]:
input_song_name = "I'm Good (Blue)"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
recommendations

                        Track Name  \
79                           REACT   
78                   Where You Are   
48                    Call It Love   
85           Rainfall (Praise You)   
73  Where Did You Go? (feat. MNEK)   

                                       Artists                     Album Name  \
79  Switch Disco, Ella Henderson, Robert Miles                          REACT   
78                          John Summit, Hayla                  Where You Are   
48                     Felix Jaehn, Ray Dalton                   Call It Love   
85                                   Tom Santa          Rainfall (Praise You)   
73                             Jax Jones, MNEK  Where Did You Go (feat. MNEK)   

   Release Date  Popularity  
79   2023-01-13          76  
78   2023-03-03          76  
48   2022-09-16          80  
85   2022-02-18          75  
73   2022-02-04          77  
Hybrid recommended songs for 'I'm Good (Blue)':


,Track Name,Artists,Album Name,Release Date,Popularity
2,Call It Love,"Felix Jaehn, Ray Dalton",Call It Love,2022-09-16,80.0
4,Where Did You Go? (feat. MNEK),"Jax Jones, MNEK",Where Did You Go (feat. MNEK),2022-02-04,77.0
0,REACT,"Switch Disco, Ella Henderson, Robert Miles",REACT,2023-01-13,76.0
1,Where You Are,"John Summit, Hayla",Where You Are,2023-03-03,76.0
3,Rainfall (Praise You),Tom Santa,Rainfall (Praise You),2022-02-18,75.0
